<a href="https://colab.research.google.com/github/DSEI21000-S21/project-product-price-prediction/blob/main/experiment/feature_extraction/FeatureExtraction_ItemDescription_E1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Google Drive Authen
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [86]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [50]:
# download sample data
drive.CreateFile({"id":"18Q86UkTlN2_hr5EviqMpAo9JKNE9jUsf"}).GetContentFile("sample_data.csv")

# download python functions
drive.CreateFile({"id":"1lQtmsLASMJJliF0XqIErqSaXRLkZeCWW"}).GetContentFile("extract_text_info.py")

# import functions from those python file
from extract_text_info import extract_general_text_info
from extract_text_info import extract_word_counts


In [42]:
df = pd.read_csv("sample_data.csv")

In [43]:
df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,748447,Polaroid Camera,3,Electronics/Cameras & Photography/Film Photogr...,NaN,27.0,1,Film not included
1,674932,Vintage Ashtray,3,Vintage & Collectibles/Housewares/Coaster,NaN,8.0,0,Sip n Smoke ashtray. Would make a cute gift!
2,586672,Matilda Jane weekender nwt,1,Women/Women's Handbags/Other,NaN,52.0,0,NWT. Matilda Jane Clothing weekender bag. Over...
3,846012,Red Full Lace Wig,3,Women/Women's Accessories/Hair Accessories,NaN,69.0,0,12 inches
4,1026408,EQUATE NICOTINE TRANSDERMAL SYSTEM,1,Other/Daily & Travel items/Personal Care,NaN,19.0,0,13 patches left. Too strong for my husband.


In [ ]:
df.item_description

0                                      Film not included
1           Sip n Smoke ashtray. Would make a cute gift!
2      NWT. Matilda Jane Clothing weekender bag. Over...
3                                              12 inches
4            13 patches left. Too strong for my husband.
                             ...                        
995    Great used condition size 38, Dansko Clogs. Bl...
996                                   No description yet
997                                 Brand new firm price
998    Set of 2 Rae Dunn Home Bowls, brand new releas...
999                                    In good condition
Name: item_description, Length: 1000, dtype: object

In [88]:
# extract general text info
stop_words = stopwords.words('english')
extract_general_text_info(df,"item_description",stop_words )

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,upper_word_count,upper_char_count,stopword_count,punctuation_count,number_count
0,748447,Polaroid Camera,3,Electronics/Cameras & Photography/Film Photogr...,NaN,27.0,1,Film not included,0,1,1,0,0
1,674932,Vintage Ashtray,3,Vintage & Collectibles/Housewares/Coaster,NaN,8.0,0,Sip n Smoke ashtray. Would make a cute gift!,0,3,1,2,0
2,586672,Matilda Jane weekender nwt,1,Women/Women's Handbags/Other,NaN,52.0,0,NWT. Matilda Jane Clothing weekender bag. Over...,1,7,0,7,0
3,846012,Red Full Lace Wig,3,Women/Women's Accessories/Hair Accessories,NaN,69.0,0,12 inches,0,0,0,0,1
4,1026408,EQUATE NICOTINE TRANSDERMAL SYSTEM,1,Other/Daily & Travel items/Personal Care,NaN,19.0,0,13 patches left. Too strong for my husband.,0,1,3,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,581369,Dansko Clogs,3,Women/Shoes/Mules & Clogs,Dansko,45.0,1,"Great used condition size 38, Dansko Clogs. Bl...",8,22,58,17,0
996,589933,Men's BKE jeans,2,"Men/Jeans/Classic, Straight Leg",BKE,23.0,1,No description yet,0,1,1,0,0
997,607839,Lularoe L Randy & OS Leggings,1,"Women/Athletic Apparel/Pants, Tights, Leggings",NaN,45.0,1,Brand new firm price,0,1,0,0,0
998,551139,Rae Dunn Home Bowl Set of 2 Red Interior,1,Home/Kitchen & Dining/Dining & Entertaining,Rae Dunn,40.0,0,"Set of 2 Rae Dunn Home Bowls, brand new releas...",0,13,16,9,1


In [97]:
df.iloc[:,8:].sum()

upper_word_count      1929
upper_char_count     12786
stopword_count        7545
punctuation_count     5924
number_count           535
dtype: int64

**Explore various Text Feature Extraction Methods**


1.   CountVectorizer from Sklearn
2.   Tfidfvectorizer from Sklearn
3.   TfidfTransformer from Sklearn
4.   HashingVectorizer from Sklearn
5.   Gensim’s Word2Vec
 


